# 基於截斷策略的機器閱讀理解

## Step1 導入套件包

In [4]:
!pip install evaluate
!pip install datasets
!pip install transformers[torch]

In [5]:
from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer, DefaultDataCollator

## Step2 資料下載

In [6]:
# 如果可以联网，直接使用load_dataset进行加载
datasets = load_dataset("cmrc2018")
# 如果无法联网，则使用下面的方式加载数据集
# datasets = DatasetDict.load_from_disk("mrc_data")
datasets

Generating train split:   0%|          | 0/10142 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3219 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1002 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 10142
    })
    validation: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 3219
    })
    test: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 1002
    })
})

In [7]:
datasets["train"][24]

{'id': 'TRAIN_614_QUERY_2',
 'context': '江苏路街道是中国上海市长宁区下辖的一个街道办事处，位于长宁区最东部，东到镇宁路接邻静安区的静安寺街道，北到武定西路接邻静安区的曹家渡街道，南到华山路接邻徐汇区的湖南路街道，西部界限为长宁路、安西路、武夷路等。面积1.52平方公里，户籍人口5.26万人，下辖13个居委会。长宁区区政府设在该街道辖区内的愚园路（近安西路）。江苏路街道的主要街道江苏路、愚园路、长宁路、武夷路、武定西路、延安西路，均为上海公共租界越界筑路，是花园洋房集中的街区，是愚园路历史文化风貌区的主体部分。较著名的近代建筑有中西女中（今上海市第三女子中学）、王伯群及汪精卫公馆（今长宁区少年宫）、西园公寓等。江苏路、长宁路、延安西路（高架）等经过拓宽，已经形成上海市的交通干道。上海市轨道交通二号线经过该街道辖区，设有江苏路站。江苏路街道下辖歧山居委会、江苏居委会、万村居委会、南汪居委会、东浜居委会、愚三居委会、曹家堰居委会、西浜居委会、福世居委会、长新居委会、华山居委会、利西居委会、北汪居委会。',
 'question': '江苏路街道较著名的近代建筑有什么？',
 'answers': {'text': ['较著名的近代建筑有中西女中（今上海市第三女子中学）、王伯群及汪精卫公馆（今长宁区少年宫）、西园公寓等。'],
  'answer_start': [237]}}

## Step3 資料預處理

In [8]:
tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-macbert-base")
tokenizer

BertTokenizerFast(name_or_path='hfl/chinese-macbert-base', vocab_size=21128, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)

In [9]:
sample_dataset = datasets["train"].select(range(10))

In [10]:
sample_dataset[0]

{'id': 'TRAIN_186_QUERY_0',
 'context': '范廷颂枢机（，），圣名保禄·若瑟（），是越南罗马天主教枢机。1963年被任为主教；1990年被擢升为天主教河内总教区宗座署理；1994年被擢升为总主教，同年年底被擢升为枢机；2009年2月离世。范廷颂于1919年6月15日在越南宁平省天主教发艳教区出生；童年时接受良好教育后，被一位越南神父带到河内继续其学业。范廷颂于1940年在河内大修道院完成神学学业。范廷颂于1949年6月6日在河内的主教座堂晋铎；及后被派到圣女小德兰孤儿院服务。1950年代，范廷颂在河内堂区创建移民接待中心以收容到河内避战的难民。1954年，法越战争结束，越南民主共和国建都河内，当时很多天主教神职人员逃至越南的南方，但范廷颂仍然留在河内。翌年管理圣若望小修院；惟在1960年因捍卫修院的自由、自治及拒绝政府在修院设政治课的要求而被捕。1963年4月5日，教宗任命范廷颂为天主教北宁教区主教，同年8月15日就任；其牧铭为「我信天主的爱」。由于范廷颂被越南政府软禁差不多30年，因此他无法到所属堂区进行牧灵工作而专注研读等工作。范廷颂除了面对战争、贫困、被当局迫害天主教会等问题外，也秘密恢复修院、创建女修会团体等。1990年，教宗若望保禄二世在同年6月18日擢升范廷颂为天主教河内总教区宗座署理以填补该教区总主教的空缺。1994年3月23日，范廷颂被教宗若望保禄二世擢升为天主教河内总教区总主教并兼天主教谅山教区宗座署理；同年11月26日，若望保禄二世擢升范廷颂为枢机。范廷颂在1995年至2001年期间出任天主教越南主教团主席。2003年4月26日，教宗若望保禄二世任命天主教谅山教区兼天主教高平教区吴光杰主教为天主教河内总教区署理主教；及至2005年2月19日，范廷颂因获批辞去总主教职务而荣休；吴光杰同日真除天主教河内总教区总主教职务。范廷颂于2009年2月22日清晨在河内离世，享年89岁；其葬礼于同月26日上午在天主教河内总教区总主教座堂举行。',
 'question': '范廷颂是什么时候被任为主教的？',
 'answers': {'text': ['1963年'], 'answer_start': [30]}}

In [11]:
tokenized_examples = tokenizer(text=sample_dataset["question"],
                text_pair=sample_dataset["context"],
                return_offsets_mapping=True,
                max_length=512, truncation="only_second", padding="max_length")
tokenized_examples.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping'])

In [12]:
# 確認句子長度
print(tokenized_examples['input_ids'][0],len(tokenized_examples["input_ids"][0]))


[101, 5745, 2455, 7563, 3221, 784, 720, 3198, 952, 6158, 818, 711, 712, 3136, 4638, 8043, 102, 5745, 2455, 7563, 3364, 3322, 8020, 8024, 8021, 8024, 1760, 1399, 924, 4882, 185, 5735, 4449, 8020, 8021, 8024, 3221, 6632, 1298, 5384, 7716, 1921, 712, 3136, 3364, 3322, 511, 9155, 2399, 6158, 818, 711, 712, 3136, 8039, 8431, 2399, 6158, 3091, 1285, 711, 1921, 712, 3136, 3777, 1079, 2600, 3136, 1277, 2134, 2429, 5392, 4415, 8039, 8447, 2399, 6158, 3091, 1285, 711, 2600, 712, 3136, 8024, 1398, 2399, 2399, 2419, 6158, 3091, 1285, 711, 3364, 3322, 8039, 8170, 2399, 123, 3299, 4895, 686, 511, 5745, 2455, 7563, 754, 9915, 2399, 127, 3299, 8115, 3189, 1762, 6632, 1298, 2123, 2398, 4689, 1921, 712, 3136, 1355, 5683, 3136, 1277, 1139, 4495, 8039, 4997, 2399, 3198, 2970, 1358, 5679, 1962, 3136, 5509, 1400, 8024, 6158, 671, 855, 6632, 1298, 4868, 4266, 2372, 1168, 3777, 1079, 5326, 5330, 1071, 2110, 689, 511, 5745, 2455, 7563, 754, 9211, 2399, 1762, 3777, 1079, 1920, 934, 6887, 7368, 2130, 2768, 4868,

In [13]:
# 確認訓練文本 句子分類 0=第一句, 1=第二句
print(list(zip(tokenized_examples['input_ids'][0], tokenized_examples['token_type_ids'][0])))


[(101, 0), (5745, 0), (2455, 0), (7563, 0), (3221, 0), (784, 0), (720, 0), (3198, 0), (952, 0), (6158, 0), (818, 0), (711, 0), (712, 0), (3136, 0), (4638, 0), (8043, 0), (102, 0), (5745, 1), (2455, 1), (7563, 1), (3364, 1), (3322, 1), (8020, 1), (8024, 1), (8021, 1), (8024, 1), (1760, 1), (1399, 1), (924, 1), (4882, 1), (185, 1), (5735, 1), (4449, 1), (8020, 1), (8021, 1), (8024, 1), (3221, 1), (6632, 1), (1298, 1), (5384, 1), (7716, 1), (1921, 1), (712, 1), (3136, 1), (3364, 1), (3322, 1), (511, 1), (9155, 1), (2399, 1), (6158, 1), (818, 1), (711, 1), (712, 1), (3136, 1), (8039, 1), (8431, 1), (2399, 1), (6158, 1), (3091, 1), (1285, 1), (711, 1), (1921, 1), (712, 1), (3136, 1), (3777, 1), (1079, 1), (2600, 1), (3136, 1), (1277, 1), (2134, 1), (2429, 1), (5392, 1), (4415, 1), (8039, 1), (8447, 1), (2399, 1), (6158, 1), (3091, 1), (1285, 1), (711, 1), (2600, 1), (712, 1), (3136, 1), (8024, 1), (1398, 1), (2399, 1), (2399, 1), (2419, 1), (6158, 1), (3091, 1), (1285, 1), (711, 1), (3364, 

In [14]:
# offset_mapping = 將字符的位置進行標示
print(tokenized_examples["offset_mapping"][0], len(tokenized_examples["offset_mapping"][0]))


[(0, 0), (0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9), (9, 10), (10, 11), (11, 12), (12, 13), (13, 14), (14, 15), (0, 0), (0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9), (9, 10), (10, 11), (11, 12), (12, 13), (13, 14), (14, 15), (15, 16), (16, 17), (17, 18), (18, 19), (19, 20), (20, 21), (21, 22), (22, 23), (23, 24), (24, 25), (25, 26), (26, 27), (27, 28), (28, 29), (29, 30), (30, 34), (34, 35), (35, 36), (36, 37), (37, 38), (38, 39), (39, 40), (40, 41), (41, 45), (45, 46), (46, 47), (47, 48), (48, 49), (49, 50), (50, 51), (51, 52), (52, 53), (53, 54), (54, 55), (55, 56), (56, 57), (57, 58), (58, 59), (59, 60), (60, 61), (61, 62), (62, 63), (63, 67), (67, 68), (68, 69), (69, 70), (70, 71), (71, 72), (72, 73), (73, 74), (74, 75), (75, 76), (76, 77), (77, 78), (78, 79), (79, 80), (80, 81), (81, 82), (82, 83), (83, 84), (84, 85), (85, 86), (86, 87), (87, 91), (91, 92), (92, 93), (93, 94), (94, 95), (95, 96), (96, 97), (97, 98), (98, 99), (

In [15]:
offset_mapping = tokenized_examples.pop("offset_mapping")


In [28]:
for idx, offset in enumerate(offset_mapping):
    answer = sample_dataset[idx]["answers"]
    start_char = answer["answer_start"][0]
    end_char = start_char + len(answer["text"][0])
    # 定位答案在token中的起始位置和结束位置
    context_start = tokenized_examples.sequence_ids(idx).index(1)
    context_end = tokenized_examples.sequence_ids(idx).index(None, context_start) - 1

    print(answer, start_char, end_char, context_start, context_end)



{'text': ['1963年'], 'answer_start': [30]} 30 35 17 510
{'text': ['1990年被擢升为天主教河内总教区宗座署理'], 'answer_start': [41]} 41 62 15 510
{'text': ['范廷颂于1919年6月15日在越南宁平省天主教发艳教区出生'], 'answer_start': [97]} 97 126 15 510
{'text': ['1994年3月23日，范廷颂被教宗若望保禄二世擢升为天主教河内总教区总主教并兼天主教谅山教区宗座署理'], 'answer_start': [548]} 548 598 17 510
{'text': ['范廷颂于2009年2月22日清晨在河内离世'], 'answer_start': [759]} 759 780 12 510
{'text': ['《全美超级模特儿新秀大赛》第十季'], 'answer_start': [26]} 26 42 21 510
{'text': ['有前途的新面孔'], 'answer_start': [247]} 247 254 20 510
{'text': ['《Jet》、《东方日报》、《Elle》等'], 'answer_start': [706]} 706 726 20 510
{'text': ['售货员'], 'answer_start': [202]} 202 205 18 510
{'text': ['大空翼'], 'answer_start': [84]} 84 87 21 486


In [17]:
answer = sample_dataset[1]["answers"]
answer

for idx, offset in enumerate(offset_mapping):
    print(idx, offset)
    break

0 [(0, 0), (0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9), (9, 10), (10, 11), (11, 12), (12, 13), (13, 14), (14, 15), (0, 0), (0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9), (9, 10), (10, 11), (11, 12), (12, 13), (13, 14), (14, 15), (15, 16), (16, 17), (17, 18), (18, 19), (19, 20), (20, 21), (21, 22), (22, 23), (23, 24), (24, 25), (25, 26), (26, 27), (27, 28), (28, 29), (29, 30), (30, 34), (34, 35), (35, 36), (36, 37), (37, 38), (38, 39), (39, 40), (40, 41), (41, 45), (45, 46), (46, 47), (47, 48), (48, 49), (49, 50), (50, 51), (51, 52), (52, 53), (53, 54), (54, 55), (55, 56), (56, 57), (57, 58), (58, 59), (59, 60), (60, 61), (61, 62), (62, 63), (63, 67), (67, 68), (68, 69), (69, 70), (70, 71), (71, 72), (72, 73), (73, 74), (74, 75), (75, 76), (76, 77), (77, 78), (78, 79), (79, 80), (80, 81), (81, 82), (82, 83), (83, 84), (84, 85), (85, 86), (86, 87), (87, 91), (91, 92), (92, 93), (93, 94), (94, 95), (95, 96), (96, 97), (97, 98), (98, 99),

In [18]:
tokenized_examples.sequence_ids(1).index(1)

15

In [19]:
for idx, offset in enumerate(offset_mapping):
    answer = sample_dataset[idx]["answers"]
    start_char = answer["answer_start"][0] # 答案起始位置
    end_char = start_char + len(answer["text"][0]) # 答案結束位置
    # 定位答案在token中的起始位置和结束位置
    # 一种策略，我们要拿到context的起始和结束，然后从左右两侧向答案逼近

    context_start = tokenized_examples.sequence_ids(idx).index(1) # 文本的起始位置
    context_end = tokenized_examples.sequence_ids(idx).index(None, context_start) - 1 # 文本結束位置

    # 判断答案是否在context中
    # offset[context_end][1] -> 文本的最後一個字
    # offset[context_start][0] -> 文本的第一個字

    if offset[context_end][1] < start_char or offset[context_start][0] > end_char:
        start_token_pos = 0
        end_token_pos = 0

    else:
        token_id = context_start
        while token_id <= context_end and offset[token_id][0] < start_char:
            token_id += 1
        start_token_pos = token_id
        token_id = context_end
        while token_id >= context_start and offset[token_id][1] > end_char:
            token_id -=1
        end_token_pos = token_id

    print(answer, start_char, end_char, context_start, context_end, start_token_pos, end_token_pos)
    print("token answer decode:", tokenizer.decode(tokenized_examples["input_ids"][idx][start_token_pos: end_token_pos + 1]))




{'text': ['1963年'], 'answer_start': [30]} 30 35 17 510 47 48
token answer decode: 1963 年
{'text': ['1990年被擢升为天主教河内总教区宗座署理'], 'answer_start': [41]} 41 62 15 510 53 70
token answer decode: 1990 年 被 擢 升 为 天 主 教 河 内 总 教 区 宗 座 署 理
{'text': ['范廷颂于1919年6月15日在越南宁平省天主教发艳教区出生'], 'answer_start': [97]} 97 126 15 510 100 124
token answer decode: 范 廷 颂 于 1919 年 6 月 15 日 在 越 南 宁 平 省 天 主 教 发 艳 教 区 出 生
{'text': ['1994年3月23日，范廷颂被教宗若望保禄二世擢升为天主教河内总教区总主教并兼天主教谅山教区宗座署理'], 'answer_start': [548]} 548 598 17 510 0 0
token answer decode: [CLS]
{'text': ['范廷颂于2009年2月22日清晨在河内离世'], 'answer_start': [759]} 759 780 12 510 0 0
token answer decode: [CLS]
{'text': ['《全美超级模特儿新秀大赛》第十季'], 'answer_start': [26]} 26 42 21 510 47 62
token answer decode: 《 全 美 超 级 模 特 儿 新 秀 大 赛 》 第 十 季
{'text': ['有前途的新面孔'], 'answer_start': [247]} 247 254 20 510 232 238
token answer decode: 有 前 途 的 新 面 孔
{'text': ['《Jet》、《东方日报》、《Elle》等'], 'answer_start': [706]} 706 726 20 510 0 0
token answer decode: [CLS]
{'text': ['售货员'], 'answer_start': [202]}

In [20]:
def process_func(examples):
    tokenized_examples = tokenizer(text=examples["question"],
                               text_pair=examples["context"],
                               return_offsets_mapping=True,
                               max_length=384, truncation="only_second", padding="max_length")
    offset_mapping = tokenized_examples.pop("offset_mapping")
    start_positions = []
    end_positions = []
    for idx, offset in enumerate(offset_mapping):
        answer = examples["answers"][idx]
        start_char = answer["answer_start"][0]
        end_char = start_char + len(answer["text"][0])
        # 定位答案在token中的起始位置和结束位置
        # 一种策略，我们要拿到context的起始和结束，然后从左右两侧向答案逼近
        context_start = tokenized_examples.sequence_ids(idx).index(1)
        context_end = tokenized_examples.sequence_ids(idx).index(None, context_start) - 1
        # 判断答案是否在context中
        if offset[context_end][1] < start_char or offset[context_start][0] > end_char:
            start_token_pos = 0
            end_token_pos = 0
        else:
            token_id = context_start
            while token_id <= context_end and offset[token_id][0] < start_char:
                token_id += 1
            start_token_pos = token_id
            token_id = context_end
            while token_id >= context_start and offset[token_id][1] > end_char:
                token_id -=1
            end_token_pos = token_id
        start_positions.append(start_token_pos)
        end_positions.append(end_token_pos)

    tokenized_examples["start_positions"] = start_positions
    tokenized_examples["end_positions"] = end_positions
    return tokenized_examples


In [21]:
tokenied_datasets = datasets.map(process_func, batched=True, remove_columns=datasets["train"].column_names)
tokenied_datasets

Map:   0%|          | 0/10142 [00:00<?, ? examples/s]

Map:   0%|          | 0/3219 [00:00<?, ? examples/s]

Map:   0%|          | 0/1002 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 10142
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 3219
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 1002
    })
})

## Step4 下載模型

In [22]:
model = AutoModelForQuestionAnswering.from_pretrained("hfl/chinese-macbert-base")

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at hfl/chinese-macbert-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Step5 配置TrainingArguments

In [23]:
args = TrainingArguments(
    output_dir="models_for_qa",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_steps=50,
    num_train_epochs=3
)

## Step6 配置Trainer

In [24]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenied_datasets["train"],
    eval_dataset=tokenied_datasets["validation"],
    data_collator=DefaultDataCollator()
)


## Step7 模型訓練

In [29]:
trainer.train()

  0%|          | 0/951 [00:00<?, ?it/s]

{'loss': 2.2894, 'learning_rate': 4.737118822292324e-05, 'epoch': 0.16}
{'loss': 1.6536, 'learning_rate': 4.474237644584648e-05, 'epoch': 0.32}
{'loss': 1.5014, 'learning_rate': 4.211356466876972e-05, 'epoch': 0.47}
{'loss': 1.4184, 'learning_rate': 3.9484752891692956e-05, 'epoch': 0.63}
{'loss': 1.3649, 'learning_rate': 3.6855941114616195e-05, 'epoch': 0.79}
{'loss': 1.3963, 'learning_rate': 3.4227129337539433e-05, 'epoch': 0.95}


  0%|          | 0/101 [00:00<?, ?it/s]

{'eval_loss': 1.0760481357574463, 'eval_runtime': 17.4063, 'eval_samples_per_second': 184.933, 'eval_steps_per_second': 5.802, 'epoch': 1.0}
{'loss': 1.0537, 'learning_rate': 3.159831756046267e-05, 'epoch': 1.1}
{'loss': 0.9732, 'learning_rate': 2.8969505783385907e-05, 'epoch': 1.26}
{'loss': 0.9436, 'learning_rate': 2.6340694006309152e-05, 'epoch': 1.42}
{'loss': 0.9087, 'learning_rate': 2.3711882229232387e-05, 'epoch': 1.58}
{'loss': 0.9384, 'learning_rate': 2.1083070452155626e-05, 'epoch': 1.74}
{'loss': 0.9478, 'learning_rate': 1.8454258675078864e-05, 'epoch': 1.89}


  0%|          | 0/101 [00:00<?, ?it/s]

{'eval_loss': 1.0511910915374756, 'eval_runtime': 17.4079, 'eval_samples_per_second': 184.916, 'eval_steps_per_second': 5.802, 'epoch': 2.0}
{'loss': 0.8321, 'learning_rate': 1.5825446898002103e-05, 'epoch': 2.05}
{'loss': 0.6253, 'learning_rate': 1.3196635120925343e-05, 'epoch': 2.21}
{'loss': 0.6351, 'learning_rate': 1.056782334384858e-05, 'epoch': 2.37}
{'loss': 0.6125, 'learning_rate': 7.93901156677182e-06, 'epoch': 2.52}
{'loss': 0.5842, 'learning_rate': 5.310199789695059e-06, 'epoch': 2.68}
{'loss': 0.598, 'learning_rate': 2.6813880126182968e-06, 'epoch': 2.84}
{'loss': 0.6177, 'learning_rate': 5.257623554153523e-08, 'epoch': 3.0}


  0%|          | 0/101 [00:00<?, ?it/s]

{'eval_loss': 1.2302634716033936, 'eval_runtime': 17.426, 'eval_samples_per_second': 184.724, 'eval_steps_per_second': 5.796, 'epoch': 3.0}
{'train_runtime': 521.6556, 'train_samples_per_second': 58.326, 'train_steps_per_second': 1.823, 'train_loss': 1.046597376105412, 'epoch': 3.0}


TrainOutput(global_step=951, training_loss=1.046597376105412, metrics={'train_runtime': 521.6556, 'train_samples_per_second': 58.326, 'train_steps_per_second': 1.823, 'train_loss': 1.046597376105412, 'epoch': 3.0})

## Step8 模型預測

In [30]:
from transformers import pipeline

pipe = pipeline("question-answering", model=model, tokenizer=tokenizer, device=0)
pipe

In [31]:
pipe(question="小明在哪里上班？", context="小明在北京上班。")

{'score': 0.8924362659454346, 'start': 3, 'end': 5, 'answer': '北京'}